# Badanie Korelacji i selekcja cech

In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

from tqdm import tqdm
import math


In [26]:
XY = pd.read_csv("sp500/data_xy.csv").set_index(keys=['quarter', 'ticker'])
CREDIT_DATA = pd.read_csv('credit_data/credit_data.csv').set_index(keys='quarter', drop=True)
XY = XY.join(CREDIT_DATA, on='quarter', how='left')

decision_attrs = [
    "price",
    "log_return_q1",
    "log_return_q2",
    "log_return_q3",
    "log_return_q4",
    "diff_q1",
    "diff_q2",
    "diff_q3",
    "diff_q4",
    "class_1_q1",
    "class_1_q2",
    "class_1_q3",
    "class_1_q4",
    "class_2_q1",
    "class_2_q2",
    "class_2_q3",
    "class_2_q4",
]

features = list(set(XY.columns) - set(decision_attrs))
features =  [f for f in features if not any([f.startswith(attr) for attr in decision_attrs])]

signals = [
    "price",
    "log_return_q1",
    "log_return_q2",
    "log_return_q3",
    "log_return_q4",
    "diff_q1",
    "diff_q2",
    "diff_q3",
    "diff_q4",
]

signal_to_decision_attrs = [
    ("log_return_q1", "class_1_q1"),
    ("log_return_q2", "class_1_q2"),
    ("log_return_q3", "class_1_q3"),
    ("log_return_q4", "class_1_q4"),
    ("diff_q1", "class_2_q1"),
    ("diff_q2", "class_2_q2"),
    ("diff_q3", "class_2_q3"),
    ("diff_q4", "class_2_q4"),
]

In [27]:
import pandas as pd
import numpy as np
import os

def safe_arctanh(x, epsilon=1e-7):
    x = np.asarray(x)
    x = np.clip(x, -1 + epsilon, 1 - epsilon)
    return np.arctanh(x)



def stream_average_correlation_matrices(XY):
    # Initialize accumulators
    all_columns = set()
    z_sum_dict = {}
    nan_count_dict = {}
    
    # First pass: collect all unique columns
    for ticker in tqdm(XY.index.get_level_values('ticker').unique()):
        file_path = f'sp500/correlation_matrixes_2/{ticker}_spearman-2.csv'
        #if os.path.exists(file_path):
        #    matrix = pd.read_csv(file_path).set_index('Unnamed: 0', drop=True)
        if True:
            ticker_data = XY.xs(ticker, level='ticker')
            matrix = ticker_data.dropna(axis=1)\
            .reset_index()\
            .drop(columns=['sector', 'sub_industry']).corr(method='spearman')
            matrix.to_csv(file_path)

    
        all_columns.update(matrix.columns)
        all_columns.update(matrix.index)
    
    all_columns = sorted(all_columns)
    
    # Process matrices one at a time
    total_matrices = 0
    for ticker in tqdm(XY.index.get_level_values('ticker').unique()):

        file_path = f'sp500/correlation_matrixes_2/{ticker}_spearman-2.csv'
        matrix = pd.read_csv(file_path).set_index("Unnamed: 0")
        total_matrices += 1
        
        z_matrix = safe_arctanh(matrix.values)
        
        for i, row_label in enumerate(matrix.index):
            for j, col_label in enumerate(matrix.columns):
                key = (row_label, col_label)
                z_value = z_matrix[i, j]
                
                if key not in z_sum_dict:
                    z_sum_dict[key] = 0
                    nan_count_dict[key] = 0
                
                if np.isnan(z_value):
                    nan_count_dict[key] += 1
                else:
                    z_sum_dict[key] += z_value
    
    # Create final matrices
    n_cols = len(all_columns)
    avg_matrix = np.full((n_cols, n_cols), np.nan)
    nan_count_matrix = np.zeros((n_cols, n_cols))
    
    # Calculate averages
    for i, row_label in enumerate(all_columns):
        for j, col_label in enumerate(all_columns):
            key = (row_label, col_label)
            if key in z_sum_dict:
                valid_count = total_matrices - nan_count_dict[key]
                if valid_count > 0:
                    avg_matrix[i, j] = np.tanh(z_sum_dict[key] / valid_count)
                nan_count_matrix[i, j] = nan_count_dict[key]
    
    # Convert to DataFrames
    avg_df = pd.DataFrame(avg_matrix, index=all_columns, columns=all_columns)
    nan_count_df = pd.DataFrame(nan_count_matrix, index=all_columns, columns=all_columns)
    
    return avg_df, nan_count_df

avg_df, nan_count_df = stream_average_correlation_matrices(XY.dropna(subset=['diff_q4', 'gross_margin']))


100%|██████████| 483/483 [00:28<00:00, 16.79it/s]


In [28]:
avg_df.to_csv("sp500/z-score-averaged-correlation-matrix-spearman-3.csv")

In [29]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(100,100))
sns.heatmap(avg_df)

<Axes: >

In [32]:
base = [
    'roe',
    'operating_margin',
    'free_cash_flow_per_share',
    'operating_cash_flow_per_share',
    'gross_margin',
    'roa',
    'asset_turnover',
    'net_profit_margin',
    'revenue',
    'operating_income',
    'cash_flow_from_operating_activities',
    'ebitda',
    #'inventory_turnover',
    'debt_equity_ratio',
    'net_income',
    'current_ratio',
    'long_term_debt_capital',
    #'receiveable_turnover',
    'retained_earnings_accumulated_deficit',
    'roi',
    'ebit_margin',
    'book_value_per_share',
    #'days_sales_in_receivables',
    'pre_tax_profit_margin',
    'total_current_assets',
    'cash_on_hand',
    'long_term_debt',
    'total_liabilities',
    'eps_earnings_per_share_diluted'
]

additional_attrs = list(set(XY.columns) - set(base))

sector_attrs = ["sector", "sub_industry"]

In [33]:
avg_df = pd.read_csv("sp500/z-score-averaged-correlation-matrix-spearman-2.csv").set_index("Unnamed: 0", drop=True)

In [45]:
import pandas as pd
import utils


for signal, decision_attr in signal_to_decision_attrs[::-1]:
    print(signal, decision_attr)
    threshold=0.01
    collinearity_threshold=0.5
    _cols = sorted(list(set(avg_df.columns).intersection(set(additional_attrs)) - set(decision_attrs))+ [signal])
    selected = utils.select_features_correlation(
        avg_df.loc[_cols, _cols],
        target_column=signal,
        threshold=0.07,
        collinearity_threshold=0.7,
        abs_values=True
    )
    features = base + selected
    
    print(len(selected))
    print(len(features))

    with open(f"feature_selection_for_{decision_attr}-selected.txt", "w") as f:
        for attr in features:
            f.write(attr+"\n")

diff_q4 class_2_q4
11
37
diff_q3 class_2_q3
11
37
diff_q2 class_2_q2
9
35
diff_q1 class_2_q1
4
30
log_return_q4 class_1_q4
10
36
log_return_q3 class_1_q3
8
34
log_return_q2 class_1_q2
8
34
log_return_q1 class_1_q1
6
32
